In [182]:
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sys
import os
from sklearn.preprocessing import MinMaxScaler

import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)

In [183]:
def convert_df_to_numpy(df, scaler):
  y = df['label'].to_numpy().astype(np.int32)
  X = scaler.transform(df.drop(columns=['label']).to_numpy())
  return X, y

center_surround_train_df = pd.read_csv("./center_surround_train.csv")
center_surround_test_df = pd.read_csv("./center_surround_test.csv")
center_surround_valid_df = pd.read_csv("./center_surround_valid.csv")

center_surround_scaler = StandardScaler()
center_surround_scaler.fit(center_surround_train_df.drop(columns=['label']).to_numpy())

center_surround_train_data = convert_df_to_numpy(center_surround_train_df, center_surround_scaler)
center_surround_test_data = convert_df_to_numpy(center_surround_test_df, center_surround_scaler)
center_surround_valid_data = convert_df_to_numpy(center_surround_valid_df, center_surround_scaler)

print(center_surround_train_data[0][:5])
print(center_surround_train_data[1][:5])

[[-1.36846388  1.54362905]
 [ 0.13476095 -0.35808876]
 [-1.17397245  2.0546884 ]
 [ 0.12212715 -0.3837286 ]
 [-0.08095132 -1.53810042]]
[1 0 1 0 1]


In [184]:
INPUT_DIM = 2
OUTPUT_DIM = 1
HIDDEN_NODES = 2
LEARNING_RATE = 0.01

In [185]:
class FeedForwardNeuralNetwork:
  def __init__(self, train_data, test_data, valid_data, input_dim, output_dim, hidden_nodes, learning_rate, hidden_activation="sigmoid", output_activation='sigmoid'):
    self.train_data = train_data
    self.test_data = test_data
    self.valid_data = valid_data
    self.input_dim = input_dim
    self.output_dim = output_dim
    self.hidden_nodes = hidden_nodes
    self.learning_rate = learning_rate
    self.hidden_activation = self.sigmoid if hidden_activation == "sigmoid" else None 
    self.output_activation = self.sigmoid if output_activation == "sigmoid" else None 

    self.W_input_hidden = None
    self.W_hidden_output = None
    self.B_input_hidden = None
    self.B_hidden_output = None

    self.initialize_weights()
    self.train()
  
  def sigmoid(self, x):
    return 1/(1 + np.exp(-x))
  
  def derivative_sigmoid(self, x):
    return self.sigmoid(x) * (1 - self.sigmoid(x))
  
  def cross_entropy_loss(self, predictions, actuals):
    loss = 0.0
    for i in range(predictions.shape[0]):
        actual = actuals[i]
        pred = predictions[i]
        internal = actual * np.log(pred) + (1 - actual) * np.log(1 - pred)
        loss += internal  
    return -loss / predictions.shape[0]
  
  def derivative_entropy_loss(self, predictions, actuals):
    gradients = np.zeros((predictions.shape[0], ))

    for i in range(len(predictions)):
      y = actuals[i]
      p = predictions[i]
      if y == 1:
          gradients[i] = -1 / p 
      else:
          gradients[i] = -(1 / (1 - p)) 
    return gradients

  def initialize_weights(self):
    """
    Initialize weights and biases for input-hidden and hidden-output layers.

    Shapes:
      Input -> Hidden:  (2, 4) weights
      Hidden -> Output: (4, 2) weights
    """
    self.W_input_hidden = np.random.randn(self.input_dim, self.hidden_nodes)
    self.W_hidden_output = np.random.randn(self.hidden_nodes, self.output_dim)
    return

  def forward(self):
    """
    Perform forward pass through hidden and output layers with activations.

    Output (first 3 points):
      [[0.69484065]
      [0.74270986]
      [0.72185353]]
    """
    X = self.train_data[0]
    self.n_q = np.dot(X, self.W_input_hidden)
    self.n_h = self.sigmoid(self.n_q)
    self.n_z = np.dot(self.n_h, self.W_hidden_output)
    self.n_p = self.sigmoid(self.n_z)
    return self.n_p

  def backpropogate(self, predictions, target):
    pass

  def train(self):
    for epoch in range(20):
      predictions = self.forward()
      actuals = self.train_data[1]
      
      error = self.cross_entropy_loss(predictions, actuals)
      print(f"Epoch ({epoch}/{20}) --- Training Error: ", error.item())

      self.backpropogate(predictions, actuals)


FeedForwardNeuralNetwork(
  train_data=center_surround_train_data,
  test_data=center_surround_test_data,
  valid_data=center_surround_valid_data,
  input_dim=INPUT_DIM,
  output_dim=OUTPUT_DIM,
  hidden_nodes=HIDDEN_NODES,
  learning_rate=LEARNING_RATE
)

**Entered the weight and biases initialization**
Neuron q:  (200, 2)
Neuron h:  (200, 2)
Neuron z:  (200, 1)
Neuron p:  (200, 1)
Epoch (0/20) --- Training Error:  0.6981130235889921


ValueError: shapes (2,200) and (1,) not aligned: 200 (dim 1) != 1 (dim 0)